# Interface description

There are three main areas in this application: (1) a [**top toolbar**](#Toolbar) with icons and with the functionality of switch the (2) **central main frame**, and a (3) set of [**lateral tabs**](#Widgets) with specific functionalities.

<img src='images/screenshot-01.1.png'></img>

## Toolbar

### Development
![](images/toolbar-dev.png)  
Consist in the area with a built-in development interface, this view is complemented with the [Projects widget](#Projects) and both create a file manager and a debug interface.

### Visualizations and Stimuli delivery
![](images/toolbar-vis.png) ![](images/toolbar-sti.png)  
This button switch to an area for display data visualizations or run a stimuli delivery, in this area is possible to execute all user development extensions. 

### Documentation
![](images/toolbar-doc.png)  
Offline documentation 

### Home
![](images/toolbar-home.png)  
Home screen with quick access for common actions.

## Tabs

### Montage
This tab is used for configure the electrodes distribution, is based in [mne.Info](https://mne.tools/stable/generated/mne.Info.html?highlight=info#mne.Info.set_montage) and is used e.g. for create topoplots. Is possible to save a custom montage, and reuse it or load one from a set of pre-configured montages. The Daisy board is configured automatically depending on the montage selected.

### Connection
This tab handle basically all [Cyton board](https://shop.openbci.com/products/cyton-biosensing-board-8-channel?variant=38958638542) hardware configuration through [OpenBCI-Stream](https://openbci-stream.readthedocs.io/en/latest/index.html), also allows to connect using any interface, Serial or WiFi, and through a distributed acquisition or not (localhost).

### Records
The records can be initialized only when an [acquisition](03-prepare_acquistion.ipynb) is active. Once you have a recording is possible to "play" the signal to simulate a stream and perform offline analysis.

### Annotations
This tab allows visualize streamed [markers, annotations and commands](83-markers_annotations_and_commands.ipynb) in real-time, or create and transmit your own.

### Extensions
This tab is used for develop and debug [Data analysis](70-develop_data_analysis.ipynb), [Visualizations](75-develop_visualizations.ipynb) and [Stimuli deliveries](80-develop_stimuli_delivery.ipynb).

## Montage

**BCI-framework** uses the [MNE standard montages](https://mne.tools/stable/generated/mne.channels.make_standard_montage.html?highlight=montage#mne.channels.make_standard_montage) through a graphical interface to pair the channel name and the OpenBCI electrode location. All this information is automatically available through any interface and can be consulted on execution time from any extension: [Stimuli delivery](80-develop_stimuli_delivery.ipynb#properties), [Data analysis](70-develop_data_analysis.ipynb#properties), and [Data visualizations](75-develop_visualizations.ipynb#properties).

<img src='images/montage.gif'></img>

## Connection

The board can be configured through this graphical interface.

<img src='images/connection.gif'></img>

**Connection:**  
Serial device through USB dongle and WiFi through WiFi shield.

**IP:**  
Port: Serial port address, like /dev/ttyUSB0 or COM0  
IP: Ip address for WiFi shield, it could be in master or slave mode.


**Latency (under TCP):**  
The time in microseconds (us) between packet sends. The higher the OpenBCI sample rate, the higher the latency needed. Default is **1000us**, minimum stable is **50us**. For upper limit sample rates such as 4kHz/8kHz/16kHz, latency around **20000us** seems to really stabilize the system.

**Host:**  
On <a href="https://openbci-stream.readthedocs.io/en/latest/notebooks/A4-server-based_acquisition.html">distributed acquisition</a> this is the IP from machine where Cyton board is connected, if the board is connected in this same machine, the the IP is just localhost.

**Acquisition sample rate:**  
Over serial connections only is available 250 samples per second, over WiFi up to 16k with 8 channels, and 8k for 16 channels.

**Streaming package size:**  
The stream will attempt to write packages of this size, a lower value will reduce the latency for real-time but increase the computational processing. A high value will be easy to proceess but will have a big delay.

**Boardmode:**
  * _Default:_ Sends accelerometer data in aux bytes
  * _Debug:_ Sends serial output over the external serial port which is helpful for debugging.
  * _Analog:_ Reads from analog pins A5(D11), A6(D12) and if no wifi shield is present, then A7(D13) as well.
  * _Digital:_ Reads from analog pins D11, D12 and D17. If no wifi present then also D13 and D18.
  * _Marker:_ Turns accel off and injects markers into the stream by sending \`X where X is any char to add to the first AUX byte.

**Test signal**   
  * _GND:_ Connect to internal GND (VDD - VSS)
  * _1X SLOW:_ Connect to test signal 1xAmplitude, slow pulse
  * _1X FAST:_ Connect to test signal 1xAmplitude, fast pulse
  * _DC:_ Connect to DC signal
  * _2X SLOW:_ Connect to test signal 2xAmplitude, slow pulse
  * _2X FAST:_ Connect to test signal 2xAmplitude, fast pulse

### Channel settings
**SRB1:**  
Select to connect all channel’s N inputs to SRB1. This effects all pins, and disconnects all N inputs from the ADC: SRB1_DISCONNECT (default), SRB1_CONNECT.

**SRB2:**  
Select to connect this channel’s P input to the SRB2 pin. This closes a switch between P input and SRB2 for the given channel, and allows the P input also remain connected to the ADC: SRB2_CONNECT (default), SRB2_DISCONNECT.

**Input type:**  
Select the ADC channel input source.

**Bias:**  
Select to include the channel input in BIAS generation.


### Leadoff Impedance

**PCHAN:**
Apply a current signal of 31.2 Hz into the P channels.

**NCHAN:**  
Apply a current signal of 31.2 Hz into the N channels.


## Electrode impedances

When the streaming is started

<img src='images/impedances.gif'></img>

## Records

When the streaming is active and the system 

<img src='images/records.gif'></img>

## Annotations

Consists of simple Kafka topics that are handled in a special way by different systems. `markers` and `annotations` are defined on [OpenBCI-Stream](https://openbci-stream.readthedocs.io/en/latest/index.html), BCI-Framework introduced `commands` to communicate the outputs of [Data analysis](70-develop_data_analysis.ipynb#Commands-and-Annotations) with external actuators or systems. Moreover is possible to create custom topics and use it anywhere.

<img src='images/annotations.gif'></img>


**Markers:**

The markers are used to synchronize the the events. Are streamed with the topic `marker`.  
Can be read from _Data analysis_ and streamed from _Stimuli delivery_.

|                                  | Stream           | Receive        |
|----------------------------------|------------------|----------------|
| Data analysis and Visualizations | X                | @loop_consumer |
| Stimuli delivery                 | self.send_marker | X              |



**Annotations:**

Used for describe special situations and other issues, like in `EDF` format. Are streamed with the topic `annotation`.  
Can be streamed from _Stimuli delivery_ and _Data analysis_.

|                                  | Stream               | 
|----------------------------------|----------------------|
| Data analysis and Visualizations | self.send_annotation |         
| Stimuli delivery                 | self.send_annotation |         

**Commands:**

Used to communicate with external services or actuators. Are streamed with the topic `command`.  
Can be read from _Stimuli delivery_ and _Data analysis_, and streamed from _Data analysis_.

|                                  | Stream            | Receive             |
|----------------------------------|-------------------|---------------------|
| Data analysis and Visualizations | self.send_command | @loop_consumer      |
| Stimuli delivery                 | X                 | self.listen_command |

**Feedbacks:**